# Mutual Information (synthetic)

Эксперименты с оценкой энтропии для синтетических данных.

# Преамбула

## Библиотеки

### Tensorflow

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa

tfds.disable_progress_bar()
tf.enable_v2_behavior()

print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices()

### Math, Numpy, Scipy, Pandas

In [ ]:
import math
import numpy as np
import scipy as sp
import scipy.stats as sps
import scipy.linalg as spl
import pandas as pd

### Matplotlib, Seaborn

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Sklearn

In [ ]:
# Деревья.
from sklearn.neighbors import KernelDensity
from sklearn.neighbors import BallTree
from sklearn.neighbors import KDTree

# Метрика.
from sklearn.metrics import pairwise_distances_argmin_min

# Метод главных компонент.
from sklearn.decomposition import PCA

# Выбор модели по кросс-валидации (поиск по сетке).
from sklearn.model_selection import GridSearchCV

### Joblib

In [ ]:
from joblib import Parallel, delayed

n_jobs = 16

### OS, shutil, Json, CSV, copy

In [ ]:
import os
import shutil
import json
import csv
import copy

## Вспомогательное

In [ ]:
# Информация об опыте.
info = dict()

In [ ]:
def normalize_uint8(data, label):
    """Нормализация: `uint8` -> `float32`."""
    return tf.cast(data, tf.float32) / 255.0, label

In [ ]:
def imshow_array(array):
    """Отображение массива нормированных пикселей."""
    plt.axis('off')
    plt.imshow((255.0 * array).astype(np.uint8), cmap=plt.get_cmap("gray"), vmin=0, vmax=255)

In [ ]:
def dataset_Y_to_X(X, Y):
    """Поменять у датасета пары (X, Y) на (X, X) (нужно, например, для обучения автоэнкодера)."""
    return X, X

In [ ]:
def concave_loss(y_true, y_pred):
    """Вогнутая функция потерь, дающая более четкие изображения при обучении."""
    delta = tf.keras.backend.abs(y_true - y_pred)
    squared = tf.keras.backend.square(y_true - y_pred)
    return tf.keras.backend.mean(delta - 0.5 * squared, axis=-1)

## Google Drive

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

## Путь к папке с данными

In [ ]:
#path = "/content/drive/My Drive/Information_v2/"
path = os.path.abspath(os.getcwd()) + "/data/"

# Синтетические данные

Для первоначальных экспериментов данные синтезируются путем сэмплирования точек из некоторого распределения с последующим отображением на некоторое многообразие.

In [ ]:
dataset_dim_1 = 32 # Размерность данных 1.
latent_dim_1  = 2  # Реальная (скрытая) размерность данных 1.

dataset_dim_2 = 32 # Размерность данных 2.
latent_dim_2  = 2  # Реальная (скрытая) размерность данных 2.

final_noize_stdev = 0.0 # Стандартное отклонение шума, складываемого с выходом функции.
samples_number = 60000 # Размер выборки.
tests_number   = 10000 # Размер тестовой выборки.

In [ ]:
experiments_path = path + "mutual_information/synthetic/"

In [ ]:
%run Miscellaneous.ipynb

### Выбор случайной величины и отображения

In [ ]:
np.random.seed(42)

In [ ]:
random_variable_1 = One_Ring(less_rad = 3.0 * np.pi, bigg_rad = 4.0 * np.pi)
random_variable_2 = One_Ring(less_rad = 3.0 * np.pi, bigg_rad = 4.0 * np.pi)
#rv_id_ensemble([sps.uniform(0.0, 2.0 * np.pi), sps.uniform(0.0, 2.0 * np.pi)])

In [ ]:
mapping_1 = mapping_ensemble([mapping_segmented([mapping_circle()] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(0,8)]),
                              mapping_segmented([mapping_circle()] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(0,8)])])

mapping_2 = mapping_ensemble([mapping_segmented([mapping_circle()] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(0,8)]),
                              mapping_segmented([mapping_circle()] * 8, [(np.pi * i, np.pi * (i+1)) for i in range(0,8)])])

In [ ]:
# Проверка входной размерности.
assert latent_dim_1 == mapping_1.input_dim
assert latent_dim_2 == mapping_2.input_dim

true_mutual_information = 0.0

### Генерация набора данных

In [ ]:
# Матрица поворота и повышения размерности.
Q_1 = sps.ortho_group.rvs(dim = dataset_dim_1)
#Q_1 = np.eye(dataset_dim)
transform_1 = Q_1[:,:mapping_1.output_dim]

Q_2 = sps.ortho_group.rvs(dim = dataset_dim_2)
#Q_2 = np.eye(dataset_dim)
transform_2 = Q_2[:,:mapping_2.output_dim]

In [ ]:
def get_samples(X, mapping, dataset_dim, transform_matrix, final_noize_stdev = 0.05):
    """
    Генерация набора данных.
    """

    # Данные во внутреннем представлении.
    samples_number = X.shape[0]
    
    # Отображение шума в пространство большей размерности.
    Y = np.zeros((samples_number, dataset_dim))
    noize = sps.norm(loc=0, scale=final_noize_stdev)
    for i in range(samples_number):
        Y[i] = transform_matrix @ mapping.map(X[i]) + noize.rvs(dataset_dim)
            
    return Y

In [ ]:
#X_1 = random_variable_1.rvs(samples_number)
#X_2 = X_1#random_variable_2.rvs(samples_number)

#T_1 = random_variable_1.rvs(tests_number)
#T_2 = T_1#random_variable_2.rvs(tests_number)

X_1 = np.concatenate((np.expand_dims(sps.uniform(loc=0.0, scale=8.0 * np.pi).rvs(samples_number), 1), np.expand_dims(sps.uniform(loc=0.0, scale=4.0 * np.pi).rvs(samples_number), 1) ), 1)
X_2 = np.concatenate((np.expand_dims(sps.uniform(loc=0.0, scale=4.0 * np.pi).rvs(samples_number), 1), np.expand_dims(sps.uniform(loc=0.0, scale=8.0 * np.pi).rvs(samples_number), 1) ), 1)

X_2[:,0] += X_1[:,1]

T_1 = np.concatenate((np.expand_dims(sps.uniform(loc=0.0, scale=8.0 * np.pi).rvs(tests_number), 1), np.expand_dims(sps.uniform(loc=0.0, scale=4.0 * np.pi).rvs(tests_number), 1) ), 1)
T_2 = np.concatenate((np.expand_dims(sps.uniform(loc=0.0, scale=4.0 * np.pi).rvs(tests_number), 1), np.expand_dims(sps.uniform(loc=0.0, scale=8.0 * np.pi).rvs(tests_number), 1) ), 1)

T_2[:,0] += T_1[:,1]

samples_1 = get_samples(X_1, mapping_1, dataset_dim_1, transform_1, final_noize_stdev)
tests_1   = get_samples(T_1, mapping_1,   dataset_dim_1, transform_1, final_noize_stdev)

samples_2 = get_samples(X_2, mapping_2, dataset_dim_2, transform_2, final_noize_stdev)
tests_2   = get_samples(T_2, mapping_2,   dataset_dim_2, transform_2, final_noize_stdev)

In [ ]:
projected = np.array([samples_1[i][0:8] for i in range(1000)])

draw_pair_plot = True
if draw_pair_plot:
    pp = sns.pairplot(pd.DataFrame(projected), height = 2.0, aspect=1.6,
                      plot_kws=dict(edgecolor="k", linewidth=0.0, alpha=0.1, size=0.01, s=0.01),
                      diag_kind="kde", diag_kws=dict(shade=True))

    fig = pp.fig
    fig.subplots_adjust(top=0.93, wspace=0.3)
    t = fig.suptitle('Pairwise Plots', fontsize=14)

### Путь к результатам

In [ ]:
dataset_path = experiments_path + ("%.3e" % final_noize_stdev) + "/" + str(samples_number) + "_" + str(tests_number) + "/"

# Оценка энтропии

## Автокодировщик

Сжатие данных предлагается делать автокодировщиком.
Для архитектуры специфицируется только формат входных данных, а также размерность внутреннего представления (кодов).

In [ ]:
# РАЗМЕРНОСТЬ КОДА.
# #
# #
 
codes_dim_1 = 2
codes_dim_2 = 2

# #
# #

In [ ]:
# Число эпох для обучения.
epochs = 10000

In [ ]:
full_path = dataset_path + "autoencoders/"
os.makedirs(full_path, exist_ok=True)

In [ ]:
info['dataset_dim_1'] = dataset_dim_1
info['latent_dim_1'] = latent_dim_1

info['dataset_dim_2'] = dataset_dim_2
info['latent_dim_2'] = latent_dim_2

info['samples_number'] = samples_number
info['tests_number'] = tests_number

info['codes_dim_1'] = codes_dim_1
info['codes_dim_2'] = codes_dim_2
info['epochs'] = epochs

### Создание модели

In [ ]:
def dense_autoencoder(shape_input, dimension):
    # Инициализация весов.
    init = tf.keras.initializers.RandomNormal(stddev = 0.02)

    # Входные данные генератора / выборки.
    input_layer = tf.keras.layers.Input(shape_input)
    next_layer = input_layer
    next_layer = tf.keras.layers.GaussianNoise(0.02)(next_layer)

    # 1 блок слоёв.
    next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(64, kernel_initializer = init),
                                                  power_iterations = 8)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)

    # 2 блок слоёв.
    next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(32, kernel_initializer = init),
                                                  power_iterations = 8)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 3 блок слоёв.
    #next_layer = tf.keras.layers.Dense(8, kernel_initializer = init)(next_layer)
    #next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # Бутылочное горлышко.
    next_layer = tfa.layers.SpectralNormalization(tf.keras.layers.Dense(dimension),
                                                  power_iterations = 8)(next_layer)
    bottleneck = tf.keras.layers.Activation('tanh', name='bottleneck')(next_layer)

    # Модель кодировщика.
    encoder = tf.keras.Model(input_layer, bottleneck)

    # Начало модели декодировщика.
    input_code_layer = tf.keras.layers.Input((dimension))
    next_layer = input_code_layer

    # 3 блок слоёв.
    #next_layer = tf.keras.layers.Dense(8, kernel_initializer = init)(next_layer)
    #next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 2 блок слоёв.
    next_layer = tf.keras.layers.Dense(32, kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)

    # 1 блок слоёв.
    next_layer = tf.keras.layers.Dense(64, kernel_initializer = init)(next_layer)
    next_layer = tf.keras.layers.LeakyReLU(alpha=0.2)(next_layer)
    
    # 0 блок слоёв.
    next_layer = tf.keras.layers.Dense(shape_input[0])(next_layer) # Подразумевается, что вход - всё равно вектор.
    #next_layer = tf.keras.layers.Activation('tanh')(next_layer)
    
    output_layer = next_layer
    
    # Модель.
    decoder = tf.keras.models.Model(input_code_layer, output_layer) # Декодировщик.
    autoencoder = tf.keras.Sequential([encoder, decoder])

    # Компиляция модели.
    opt = tf.keras.optimizers.Adam(learning_rate = 5e-3)
    autoencoder.compile(loss = 'mse', optimizer = opt, loss_weights = [1.0])
    
    return encoder, decoder, autoencoder

### Загрузка модели

In [ ]:
#encoder = tf.keras.models.load_model(full_path + "encoder.h5")
#decoder = tf.keras.models.load_model(full_path + "decoder.h5")
#autoencoder = autoencoder = tf.keras.Sequential([encoder, decoder])
#autoencoder.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3), loss_weights = [1.0])

#with open(full_path + 'info.json', 'r') as fp:
#    info = json.load(fp)

In [ ]:
encoder_1, decoder_1, autoencoder_1 = dense_autoencoder((dataset_dim_1,), codes_dim_1)
encoder_2, decoder_2, autoencoder_2 = dense_autoencoder((dataset_dim_2,), codes_dim_2)

#### Обучение 1

In [ ]:
autoencoder_1.fit(samples_1, samples_1, epochs=epochs, validation_data=(tests_1, tests_1), batch_size=samples_number // 10)

In [ ]:
autoencoder_1.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3), loss_weights = [1.0])

In [ ]:
autoencoder_1.fit(samples_1, samples_1, epochs=1000, validation_data=(tests_1, tests_1), batch_size=samples_number)

#### Обучение 2

In [ ]:
autoencoder_2.fit(samples_2, samples_2, epochs=epochs, validation_data=(tests_2, tests_2), batch_size=samples_number // 10)

In [ ]:
autoencoder_2.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3), loss_weights = [1.0])

In [ ]:
autoencoder_2.fit(samples_2, samples_2, epochs=1000, validation_data=(tests_2, tests_2), batch_size=samples_number)

In [ ]:
# Сохранение моделей.
autoencoder_1.save(full_path + "autoencoder_1.h5")
encoder_1.save(full_path + "encoder_1.h5")
decoder_1.save(full_path + "decoder_1.h5")

autoencoder_2.save(full_path + "autoencoder_2.h5")
encoder_2.save(full_path + "encoder_2.h5")
decoder_2.save(full_path + "decoder_2.h5")

In [ ]:
# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

### Получение кодов всех элементов набора данных

In [ ]:
codes_1 = np.array(encoder_1.predict(samples_1))
codes_2 = np.array(encoder_2.predict(samples_2))

In [ ]:
codes_12 = np.concatenate((codes_1, codes_2), 1)
codes_dim_12 = codes_dim_1 + codes_dim_2

In [ ]:
codes_pca_dim_1 = codes_dim_1
PCA_codes_1 = PCA(n_components=codes_pca_dim_1, whiten=True)
codes_pca_1 = np.array(PCA_codes_1.fit_transform(codes_1))

codes_pca_dim_2 = codes_dim_2
PCA_codes_2 = PCA(n_components=codes_pca_dim_2, whiten=True)
codes_pca_2 = np.array(PCA_codes_2.fit_transform(codes_2))

In [ ]:
codes_pca_dim_12 = codes_dim_12
PCA_codes_12 = PCA(n_components=codes_pca_dim_12, whiten=True)
codes_pca_12 = np.array(PCA_codes_12.fit_transform(codes_12))

In [ ]:
pp = sns.pairplot(pd.DataFrame(codes_pca_12[0:10000]), height = 2.0, aspect=1.6,
                      plot_kws=dict(edgecolor="k", linewidth=0.0, alpha=0.05, size=0.01, s=0.01),
                      diag_kind="kde", diag_kws=dict(shade=True))

fig = pp.fig
fig.subplots_adjust(top=0.93, wspace=0.3)
t = fig.suptitle('Pairwise Plots', fontsize=14)

### KDE для кодов

In [ ]:
# Загрузка параметров KDE.

#with open(full_path + 'info.json', 'r') as fp:
#    info = json.load(fp)

#kde_codes = KernelDensity(bandwidth=info['bandwidth'], kernel='gaussian')
#kde_codes.fit(codes_pca)

In [ ]:
def smart_gridsearch(begin, end, data, resolution = 7, rel_x_epsilon = 0.01, rtol = 0.001, n_jobs = 2, cv = 20):
    while True:
        grid = np.logspace(np.log10(begin), np.log10(end), resolution)
        print("Поиск по сетке: ", grid)
        params = {'bandwidth': grid}
        
        grid_search = GridSearchCV(KernelDensity(rtol = rtol, kernel='gaussian'), params, n_jobs = n_jobs, verbose = 10, cv = cv)
        grid_search.fit(data)
        
        if grid_search.best_index_ == 0:
            begin *= begin / end
            end = grid[1]
        elif grid_search.best_index_ == resolution - 1:
            end *= end / begin
            begin = grid[-2]
        else:
            begin = grid[grid_search.best_index_ - 1]
            end = grid[grid_search.best_index_ + 1]

            if end - begin < rel_x_epsilon * grid[grid_search.best_index_]:
                return grid_search 

In [ ]:
KDE_codes_1 = smart_gridsearch(0.05, 0.2, codes_pca_1, n_jobs = n_jobs).best_estimator_
KDE_codes_1.set_params(rtol = 0.0)
print(KDE_codes_1.get_params())

In [ ]:
KDE_codes_2 = smart_gridsearch(0.05, 0.2, codes_pca_2, n_jobs = n_jobs).best_estimator_
KDE_codes_2.set_params(rtol = 0.0)
print(KDE_codes_2.get_params())

In [ ]:
KDE_codes_12 = smart_gridsearch(0.05, 0.2, codes_pca_12, n_jobs = n_jobs).best_estimator_
KDE_codes_12.set_params(rtol = 0.0)
#KDE_codes_12 = KernelDensity(rtol = 0.0, bandwidth = max(KDE_codes_1.get_params()['bandwidth'], KDE_codes_2.get_params()['bandwidth']))
print(KDE_codes_12.get_params())

In [ ]:
info['bandwidth_1'] = KDE_codes_1.get_params()['bandwidth']
info['bandwidth_2'] = KDE_codes_2.get_params()['bandwidth']
info['bandwidth_12'] = KDE_codes_12.get_params()['bandwidth']

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

## Подсчёт энтропии

In [ ]:
def _loo_step(bandwidth, samples, i):
    loo_samples = samples
    np.delete(loo_samples, i)
    
    kde = KernelDensity(bandwidth=bandwidth, kernel='gaussian')
    kde.fit(loo_samples)
    return kde.score_samples([samples[i]])[0]

In [ ]:
def entropy_leave_one_out_parallel(path, bandwidth, samples, n_jobs = 2, first_N = None, parts = 10, recover_saved = False):
    """
    Параллельное вычисление оценки энтропии методом убрать-один-элемент.
    """
    
    # Создание временных папок для сохранения прогресса.
    parts_path = path + "LOO_PARTS/"
    os.makedirs(parts_path, exist_ok=True)

    # Если дано first_N, энтропия будет оцениваться только на первых first_N элементах.
    N = 0
    if first_N is None:
        N = len(samples)
    else:
        N = first_N

    # Число частей и массив, их содержащий.
    N_per_part = N // parts
    log_probs = []

    # Восстанавливаем прогресс, если требуется.
    recovered_parts = 0
    if recover_saved:
        for filename in os.listdir(parts_path):
            if filename.endswith(".csv"):
                log_probs.append(np.loadtxt(parts_path + filename))
                recovered_parts += 1

    print("Восстановлено блоков данных: %d" % recovered_parts)

    # Подсчёт логарифма вероятности в точках.
    for part in range(recovered_parts, parts):
        log_probs.append(
            np.array(
                Parallel(n_jobs = n_jobs, verbose = 10, batch_size = 8)(
                    delayed(_loo_step)(bandwidth, samples, i) for i in range(part * N_per_part, min((part + 1) * N_per_part, N))
                )
            )
        )
        np.savetxt(parts_path + str(part) + ".csv", log_probs[part], delimiter="\n")
    
    # Объединение в один массив.
    log_prob = np.concatenate(log_probs)

    # Суммирование и нахождение стандартного отклонения.
    average = -math.fsum(log_prob) / N    
    squared_deviations = np.zeros(N)
    for i in range(N):
        squared_deviations[i] = (log_prob[i] - average)**2
    standard_deviation = np.sqrt(math.fsum(squared_deviations) / (N * (N - 1)))
    
    # Удаление временных файлов.
    shutil.rmtree(parts_path)
        
    return average, standard_deviation

In [ ]:
latent_entropy_1, latent_entropy_error_1 = entropy_leave_one_out_parallel(full_path, KDE_codes_1.get_params()['bandwidth'], codes_pca_1, n_jobs = n_jobs, recover_saved = False)
print("LH1: %f, errLH1: %f" % (latent_entropy_1, latent_entropy_error_1))

In [ ]:
latent_entropy_2, latent_entropy_error_2 = entropy_leave_one_out_parallel(full_path, KDE_codes_2.get_params()['bandwidth'], codes_pca_2, n_jobs = n_jobs, recover_saved = False)
print("LH2: %f, errLH2: %f" % (latent_entropy_2, latent_entropy_error_2))

In [ ]:
latent_entropy_12, latent_entropy_error_12 = entropy_leave_one_out_parallel(full_path, KDE_codes_12.get_params()['bandwidth'], codes_pca_12, n_jobs = n_jobs, recover_saved = False)
print("LH12: %f, errLH12: %f" % (latent_entropy_12, latent_entropy_error_12))

In [ ]:
info['latent_entropy_1'] = latent_entropy_1
info['latent_entropy_error_1'] = latent_entropy_error_1

info['latent_entropy_2'] = latent_entropy_2
info['latent_entropy_error_2'] = latent_entropy_error_2

info['latent_entropy_12'] = latent_entropy_12
info['latent_entropy_error_12'] = latent_entropy_error_12

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

In [ ]:
# Коэффициент растяжения при денормализации.
PCA_codes_defc_1 = np.abs(np.linalg.det( PCA_codes_1.inverse_transform(np.eye(codes_pca_dim_1)) -
                                         PCA_codes_1.inverse_transform(np.zeros((codes_pca_dim_1, codes_pca_dim_1))) ))

PCA_codes_defc_2 = np.abs(np.linalg.det( PCA_codes_2.inverse_transform(np.eye(codes_pca_dim_2)) -
                                         PCA_codes_2.inverse_transform(np.zeros((codes_pca_dim_2, codes_pca_dim_2))) ))
                                                                       
PCA_codes_defc_12 = np.abs(np.linalg.det( PCA_codes_12.inverse_transform(np.eye(codes_pca_dim_12)) -
                                          PCA_codes_12.inverse_transform(np.zeros((codes_pca_dim_12, codes_pca_dim_12))) ))

In [ ]:
# Соответствующая энтропия.
PCA_codes_transform_entropy_1 = np.log(PCA_codes_defc_1)
PCA_codes_transform_entropy_2 = np.log(PCA_codes_defc_2)
PCA_codes_transform_entropy_12 = np.log(PCA_codes_defc_12)

print("PCA_TH1: %f" % (PCA_codes_transform_entropy_1))
print("PCA_TH2: %f" % (PCA_codes_transform_entropy_2))
print("PCA_TH12: %f" % (PCA_codes_transform_entropy_12))

In [ ]:
info['PCA_codes_transform_entropy_1'] = PCA_codes_transform_entropy_1
info['PCA_codes_transform_entropy_2'] = PCA_codes_transform_entropy_2
info['PCA_codes_transform_entropy_12'] = PCA_codes_transform_entropy_12

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)

In [ ]:
# Итоговая оценка энтропии.
entropy_1 = latent_entropy_1 + PCA_codes_transform_entropy_1
entropy_error_1 = latent_entropy_error_1

entropy_2 = latent_entropy_1 + PCA_codes_transform_entropy_2
entropy_error_2 = latent_entropy_error_2

entropy_12 = latent_entropy_12 + PCA_codes_transform_entropy_12
entropy_error_12 = latent_entropy_error_12

print("H1: %f, errH1: %f\nH2: %f, errH2: %f\nH12: %f, errH12: %f" %
      (entropy_1, entropy_error_1, entropy_2, entropy_error_2, entropy_12, entropy_error_12))

In [ ]:
mutual_information = entropy_1 + entropy_2 - entropy_12
mutual_information_error = entropy_error_12 + entropy_error_1 + entropy_error_2

print("MI: %f, errMI: %f" % (mutual_information, mutual_information_error))

In [ ]:
info['entropy_1'] = entropy_1
info['entropy_error_1'] = entropy_error_1

info['entropy_2'] = entropy_2
info['entropy_error_2'] = entropy_error_2

info['entropy_12'] = entropy_12
info['entropy_error_12'] = entropy_error_12


info['mutual_information'] = mutual_information
info['mutual_information_error'] = mutual_information_error

# Сохранение информации.
with open(full_path + 'info.json', 'w') as fp:
    json.dump(info, fp, indent=4)